In [3]:
import pandas as pd
import numpy as np
import re #re stands for regular expressions.
#Regular expressions are a powerful way to search, match, and manipulate text based on patterns.
#In this code, I use it to extract numeric values from messy strings.
df = pd.read_csv(r"C:\Users\User\Downloads\Nutrition__Physical_Activity__and_Obesity_-_Behavioral_Risk_Factor_Surveillance_System.csv")
print(df.shape)
df.head()
df.info()

(106260, 33)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106260 entries, 0 to 106259
Data columns (total 33 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   YearStart                   106260 non-null  int64  
 1   YearEnd                     106260 non-null  int64  
 2   LocationAbbr                106260 non-null  object 
 3   LocationDesc                106260 non-null  object 
 4   Datasource                  106260 non-null  object 
 5   Class                       106260 non-null  object 
 6   Topic                       106260 non-null  object 
 7   Question                    106260 non-null  object 
 8   Data_Value_Unit             0 non-null       float64
 9   Data_Value_Type             106260 non-null  object 
 10  Data_Value                  93505 non-null   float64
 11  Data_Value_Alt              93505 non-null   float64
 12  Data_Value_Footnote_Symbol  12755 non-null   object 
 13  D

In [4]:
#------Checking numerical_columns and converting problemetic values into NaN------
def safe_numeric(x):   #Converts x to numeric only if it looks numeric. Leaves all other text unchanged.
    if pd.isna(x) or x == '': 
        return np.nan  #if x is missing or empty, return NaN immediately. This ensures we don’t try to process missing values — they stay as NaN
    x_clean = str(x).replace(',','').replace('%','').strip()  #it will convert "x" whatever it is into a string+remove commas+remover percentage sign+ extra empty spaces.
### Extracting Numerical Values from strings using "Regex Method"
   
    match = re.search(r'[-+]?\d*\.\d+|\d+', x_clean)  #re.search() looks for the first occurrence of the pattern in the string x_clean
    if match:
        return float(match.group()) #match.group() returns the matched string, e.g., "12.5", float(...) will convert it to a float point.
    else:
        return x
column_to_process = []
for col in df.columns:
    sample = df[col].dropna().astype(str).head(1000)
    numeric_col = sample.str.contains(r'\d').any()   #.str.contains() searches each value for a pattern.\d means "any digit" (0–9)
    if numeric_col:
        column_to_process.append(col)
print("Colunms detected for numeric conversion:", column_to_process)
df.shape

Colunms detected for numeric conversion: ['YearStart', 'YearEnd', 'Question', 'Data_Value', 'Data_Value_Alt', 'Low_Confidence_Limit', 'High_Confidence_Limit ', 'Sample_Size', 'Age(years)', 'Income', 'Race/Ethnicity', 'GeoLocation', 'TopicID', 'QuestionID', 'LocationID', 'Stratification1', 'StratificationID1']


(106260, 33)

In [5]:
# Converting non_numeric into Nan
for col in column_to_process:
    df[col] = df[col].apply(safe_numeric)
    print(f"Processed column: {col}")


Processed column: YearStart
Processed column: YearEnd
Processed column: Question
Processed column: Data_Value
Processed column: Data_Value_Alt
Processed column: Low_Confidence_Limit
Processed column: High_Confidence_Limit 
Processed column: Sample_Size
Processed column: Age(years)
Processed column: Income
Processed column: Race/Ethnicity
Processed column: GeoLocation
Processed column: TopicID
Processed column: QuestionID
Processed column: LocationID
Processed column: Stratification1
Processed column: StratificationID1


In [6]:
#------Standardise text: e.g Location names------
string_col = df.select_dtypes(include = ['object']).columns
for col in string_col:
    df[col] = df[col].astype(str).str.strip().str.title()
df.shape

(106260, 33)

In [7]:
#------handeling missing values------
print(df.isna().sum())
#dropping all columns with too many missing values
df = df.dropna(thresh = 0.7*len(df), axis = 1) #axis=1 means column_wise dropping
#dropping rows with missing values from the columns with most missing values
important_cols = ['Data_Value','Sample_Size','Low_Confidence_Limit','High_Confidence_Limit ']
df = df.dropna(subset= important_cols)
df.shape

YearStart                          0
YearEnd                            0
LocationAbbr                       0
LocationDesc                       0
Datasource                         0
Class                              0
Topic                              0
Question                           0
Data_Value_Unit               106260
Data_Value_Type                    0
Data_Value                     12755
Data_Value_Alt                 12755
Data_Value_Footnote_Symbol         0
Data_Value_Footnote                0
Low_Confidence_Limit           12755
High_Confidence_Limit          12755
Sample_Size                    12755
Total                              0
Age(years)                     83490
Education                          0
Sex                                0
Income                             0
Race/Ethnicity                     0
GeoLocation                     1932
ClassID                            0
TopicID                            0
QuestionID                         0
D

(93505, 31)

In [8]:
#------Handeling Duplicates------
df = df.drop_duplicates()
df.shape

(93505, 31)

In [9]:
#handeling outliners
# Clean column names
df.columns = df.columns.str.strip()

# Drop duplicates
df = df.drop_duplicates()

# Convert numeric columns safely
numeric_cols = ['Data_Value', 'Low_Confidence_Limit', 'High_Confidence_Limit', 'Sample_Size']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Remove outliers for numeric columns
for col in numeric_cols:
    df = df[(df[col].isna()) | ((df[col] >= 0) & (df[col] <= 100))]

print("Rows remaining:", df.shape[0])
df.shape

Rows remaining: 5222


(5222, 31)

In [10]:
#------Standardisation & Nomarlisation------

string_cols = df.select_dtypes(include=['object']).columns.tolist()
for col in string_cols:
    df[col] = df[col].astype(str).str.strip().str.lower()
   

In [11]:
#------extracting meaningful information into new columns-----
#extract age / gender / race / income / education

df.columns = df.columns.str.strip().str.lower()
df['age_group']=df['stratification1'].where(df['stratificationcategory1']=='age(years)')
df['gender']=df['stratification1'].where(df['stratificationcategory1']=='gender')
df['income_level']=df['stratification1'].where(df['stratificationcategory1']=='income')
df['education_level']= df['stratification1'].where(df['stratificationcategory1']=='education')
df['race_ethnicity'] = df['stratification1'].where(df['stratificationcategory1']=='race/ethnicity')
#cleaning new column e.g, space inbetween or casing
cols = ['age_group','gender','income_level','education_level','race_ethnicity']
for col in cols:
    df[col]= df[col].astype(str).str.strip().str.lower()
    df[col]=df[col].astype('category')
  

In [12]:
df.shape

(5222, 36)

In [13]:
# Check that sample_size is positive
df = df[df['sample_size'] > 0]


In [14]:
# Drop irrelevant columns
cols_to_drop = ['data_value_alt','footnote_symbol','geolocation']  # example
df.drop(columns=cols_to_drop, axis=1, inplace=True, errors='ignore')
df.shape

(5222, 34)

In [15]:
print(df.info())
print(df.describe())


<class 'pandas.core.frame.DataFrame'>
Index: 5222 entries, 6 to 106257
Data columns (total 34 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   yearstart                   5222 non-null   float64 
 1   yearend                     5222 non-null   float64 
 2   locationabbr                5222 non-null   object  
 3   locationdesc                5222 non-null   object  
 4   datasource                  5222 non-null   object  
 5   class                       5222 non-null   object  
 6   topic                       5222 non-null   object  
 7   question                    5222 non-null   object  
 8   data_value_type             5222 non-null   object  
 9   data_value                  5222 non-null   float64 
 10  data_value_footnote_symbol  5222 non-null   object  
 11  data_value_footnote         5222 non-null   object  
 12  low_confidence_limit        5222 non-null   float64 
 13  high_confidence_limit

In [17]:
#EXPORT CLEAN DATA
df.to_csv("BRFSS_nutrition_physical_activity_clean.csv", index=False)
print("✅ Cleaned dataset saved.")


✅ Cleaned dataset saved.
